In [2]:
from query.models import Face, Shot
from rekall.video_interval_collection import VideoIntervalCollection
from rekall.parsers import named_payload, in_array, bbox_payload_parser
from rekall.parsers import merge_dict_parsers, dict_payload_parser
from rekall.merge_ops import payload_plus, payload_first, merge_named_payload
from rekall.payload_predicates import payload_satisfies, on_name
from rekall.spatial_predicates import scene_graph
from rekall.logical_predicates import and_pred
from rekall.temporal_predicates import overlaps
from rekall.bbox_predicates import height_at_least, left_of, same_value
from esper.rekall import intrvllists_to_result_with_objects, bbox_to_result_object

In [3]:
from esper.prelude import esper_widget
import time
from IPython.display import display

# Query

In [11]:
def query(video_id, num_people, MIN_FACE_HEIGHT, MIN_BRIGHTNESS, stride):
    # We're going to look for frames that have main characters looking hopeful.
    # This is the same as hero shots, except there have to be multiple people
    # We're going to look for frames where there's exactly one face of a
    #   certain height, and the frame has certain minimum brightness,
    #   sharpness, and contrast properties.
#     MIN_FACE_HEIGHT = 0.2
#     MIN_BRIGHTNESS = 50

    # Annotate face rows with start and end frames, video ID, and frame image
    #   information
    faces_qs = Face.objects.annotate(
        min_frame=F('frame__number'),
        max_frame=F('frame__number'),
        video_id=F('frame__video_id'),
        brightness=F('frame__brightness')
    ).filter(
        frame__video_id=video_id,
        brightness__isnull=False
    )

    # Load bounding boxes and faces into rekall, and put all faces in one frame
    faces = VideoIntervalCollection.from_django_qs(
        faces_qs,
        with_payload=merge_dict_parsers([named_payload('faces',
            in_array(bbox_payload_parser(VideoIntervalCollection.django_accessor))),
            dict_payload_parser(VideoIntervalCollection.django_accessor, {
                'brightness': 'brightness'
            })])
    ).coalesce(merge_named_payload({
        'faces': payload_plus,
        'brightness': payload_first
    }))

    # Hero shots are shots where there is exactly one face of at least a
    #   certain height, and brightness, contrast, and sharpness are at least
    #   some amount
    hero_shot_frames = faces.filter(payload_satisfies(and_pred(
        and_pred(
            on_name('faces', scene_graph({
                'nodes': [{ 'name': 'face', 'predicates': [
                    height_at_least(MIN_FACE_HEIGHT) ] }],
                'edges': []
            }, exact=False)),
            on_name('faces', lambda p: len(p) >= num_people)
        ),
        lambda payload: (payload['brightness'] > MIN_BRIGHTNESS)
    )))
    
    shots_qs = Shot.objects.filter(
        video_id=video_id,
        cinematic=True)
    shots = VideoIntervalCollection.from_django_qs(shots_qs)
    
    hero_shots = shots.filter_against(hero_shot_frames, predicate=overlaps())
    
    return intrvllists_to_result_with_objects(
        hero_shots.get_allintervals(), 
        lambda a, b: [],
#         lambda payload, video_id: [
#             bbox_to_result_object(bbox, video_id) for bbox in payload['faces']],
        stride=stride)

In [12]:
def show_query(video_id, num_people, min_face_height, min_brightness, stride):
    result = query(video_id, num_people, min_face_height, min_brightness, stride)
    
    widget = esper_widget(result, jupyter_keybindings=True, crop_bboxes=False)
    display(widget)
    
    return widget, result

In [49]:
def convert_segments(segments):
    output_segments = []
    for seg in segments:
        for res in result['result']:
            if res['elements'][0]['min_frame'] == seg[0]:
                output_segments.append((res['elements'][0]['min_frame'], res['elements'][0]['max_frame']))
    print(output_segments)

# Braveheart

In [52]:
widget, result = show_query(28, 3, 0.2, 75, 5)

VGridWidget(jsglobals={'bucket': 'esper', 'queries': [['Reaction shots in Apollo 13 (rekall)', 'def reaction_s…

In [15]:
selected_segments_braveheart = [
    (result['result'][i]['elements'][0]['min_frame'], result['result'][i]['elements'][0]['min_frame'])
    for i in widget.selected
]
print(selected_segments_braveheart)

[(34768, 34768), (227434, 227434), (128001, 128001)]


In [53]:
convert_segments(selected_segments_braveheart)

[(34768, 34864), (227434, 227503), (128001, 128023)]


# Revenge of the Sith

In [16]:
start = time.time()

In [50]:
widget, result = show_query(186, 3, 0.1, 25, 1)

VGridWidget(jsglobals={'bucket': 'esper', 'queries': [['Reaction shots in Apollo 13 (rekall)', 'def reaction_s…

In [20]:
selected_segments_rots = [
    (result['result'][i]['elements'][0]['min_frame'], result['result'][i]['elements'][0]['min_frame'])
    for i in widget.selected
]
print(selected_segments_rots)

[(35910, 35910), (58774, 58774), (143737, 143737), (185812, 185812)]


In [51]:
convert_segments(selected_segments_rots)

[(35910, 36096), (58774, 58912), (143737, 143904), (185812, 185915)]


In [21]:
end = time.time()

In [22]:
print("Seconds to label: ", end - start)

Seconds to label:  90.56312465667725


# Steve Jobs

In [23]:
start = time.time()

In [54]:
widget, result = show_query(520, 3, 0.2, 50, 1)

VGridWidget(jsglobals={'bucket': 'esper', 'queries': [['Reaction shots in Apollo 13 (rekall)', 'def reaction_s…

In [25]:
selected_segments_jobs = [
    (result['result'][i]['elements'][0]['min_frame'], result['result'][i]['elements'][0]['min_frame'])
    for i in widget.selected
]
print(selected_segments_jobs)

[(13297, 13297), (43945, 43945), (159222, 159222)]


In [55]:
convert_segments(selected_segments_jobs)

[(13297, 13346), (43945, 43964), (159222, 159277)]


In [26]:
end = time.time()

In [27]:
print("Seconds to label: ", end - start)

Seconds to label:  88.11607384681702


# Guardians of the Galaxy

In [28]:
start = time.time()

In [56]:
widget, result = show_query(74, 3, 0.1, 25, 1)

VGridWidget(jsglobals={'bucket': 'esper', 'queries': [['Reaction shots in Apollo 13 (rekall)', 'def reaction_s…

In [32]:
selected_segments_gotg = [
    (result['result'][i]['elements'][0]['min_frame'], result['result'][i]['elements'][0]['min_frame'])
    for i in widget.selected
]
print(selected_segments_gotg)

[(37823, 37823), (23901, 23901), (156920, 156920), (157403, 157403)]


In [57]:
convert_segments(selected_segments_gotg)

[(37823, 38000), (23901, 23999), (156920, 156980), (157403, 157487)]


In [33]:
end = time.time()

In [34]:
print("Seconds to label: ", end - start)

Seconds to label:  126.58501887321472


# Daddy's Home

In [35]:
start = time.time()

In [58]:
widget, result = show_query(334, 3, 0.3, 60, 1)

VGridWidget(jsglobals={'bucket': 'esper', 'queries': [['Reaction shots in Apollo 13 (rekall)', 'def reaction_s…

In [40]:
selected_segments_daddy = [
    (result['result'][i]['elements'][0]['min_frame'], result['result'][i]['elements'][0]['min_frame'])
    for i in widget.selected
]
print(selected_segments_daddy)

[(33339, 33339), (34751, 34751), (96419, 96419)]


In [59]:
convert_segments(selected_segments_daddy)

[(33339, 33419), (34751, 34847), (96419, 96442)]


In [41]:
end = time.time()

In [42]:
print("Seconds to label: ", end - start)

Seconds to label:  58.61626982688904


# Batman v Superman

In [43]:
start = time.time()

In [60]:
widget, result = show_query(299, 3, 0.1, 20, 1)

VGridWidget(jsglobals={'bucket': 'esper', 'queries': [['Reaction shots in Apollo 13 (rekall)', 'def reaction_s…

In [46]:
selected_segments_bvs = [
    (result['result'][i]['elements'][0]['min_frame'], result['result'][i]['elements'][0]['min_frame'])
    for i in widget.selected
]
print(selected_segments_bvs)

[(128261, 128261), (132495, 132495), (144193, 144193), (127155, 127155)]


In [61]:
convert_segments(selected_segments_bvs)

[(128261, 128382), (132495, 132680), (144193, 144335), (127155, 127225)]


In [47]:
end = time.time()

In [48]:
print("Seconds to label: ", end - start)

Seconds to label:  105.20355749130249
